In [13]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('Cancer_Dump.csv')

/var/folders/s8/5rrw77y553vc4scyrbx8sdg40000gn/T/ipykernel_85733/3997312260.py:4: DtypeWarning: Columns (22,23,24,25,56,57,58,59,60,61,62,64,65,66,67,68,69,70,71,72,73,74,90,91,92,98,100,102,106,107,108,111,113,114,115,116,117,118,119,120,121,128,137,138,139,143,144,146,147,148,149,151,153,154) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Cancer_Dump.csv')


In [ ]:
slice_df = df[df['protocolSection.identificationModule.nctId'] == 'NCT05705401']
slice_df.to_csv('filtered_data_NCT05705401.csv', index=False)

In [50]:
import ast

def extract_intervention_types(interventions):
    # interventions가 문자열이면 리스트로 변환 시도
    if isinstance(interventions, str):
        try:
            interventions = ast.literal_eval(interventions)  # 문자열을 리스트로 변환
        except (ValueError, SyntaxError):
            return []  # 변환할 수 없는 경우 빈 리스트 반환
    
    # interventions가 리스트이면 'type' 값 추출
    if isinstance(interventions, list):
        return [item['type'] for item in interventions if isinstance(item, dict) and 'type' in item]
    
    return []  # 리스트가 아닌 경우 빈 리스트 반환


In [56]:
#1. completion date between 2008.01-2021.12.31
df['completion_date'] = pd.to_datetime(df['protocolSection.statusModule.completionDateStruct.date'], errors='coerce', infer_datetime_format=True)

# set the range of date
start_date = '2008-01-01'
end_date = '2021-12-31'

#2. Have start data
df['start_date'] = pd.to_datetime(df['protocolSection.statusModule.startDateStruct.date'], errors='coerce', infer_datetime_format=True)

#3. Have primary start date
df['primary_start_date'] = pd.to_datetime(df['protocolSection.statusModule.primaryCompletionDateStruct.date'], errors='coerce', infer_datetime_format=True)

#4. Include Phase 1 and 4
def include_phases(phases):
    if isinstance(phases, str):
        try:
            phases = ast.literal_eval(phases)
        except (ValueError, SyntaxError):
            return False  
    if isinstance(phases, list):
        return any(phase in ['PHASE2', 'PHASE3'] for phase in phases)
    
    return False

#5. Sponsor contains oncology OR trials reported results at Oncology
# protocolSection.sponsorCollaboratorsModule.leadSponsor.name
# df['sponsor'] = df['protocolSection.identificationModule.organization.fullName']

#6. Recruitment Status is Completed
df['Recruitment'] = df['protocolSection.statusModule.overallStatus']

#7. studyType is Interventional
df['Interventional']= df['protocolSection.designModule.studyType']

#8. Inverventional type is drug or biological
df['InterventionalType'] = df['protocolSection.armsInterventionsModule.interventions'].apply(extract_intervention_types)

filtered_df = df[(df['completion_date'] >= start_date) & 
                 (df['completion_date'] <= end_date) &
                 (df['start_date'].notna()) &
                 (df['primary_start_date'].notna()) &
                 (df['protocolSection.designModule.phases'].apply(include_phases)) &
                 (df['Recruitment']=='COMPLETED') &
                 (df['Interventional']=='INTERVENTIONAL')&
                 (df['InterventionalType'].apply(lambda x: 'BIOLOGICAL' in x or 'DRUG' in x)) ]

# print the number of filtered data
count = filtered_df.shape[0]
print(f"The number of data from January 2008 to December 31th 2021: {count}")

/var/folders/s8/5rrw77y553vc4scyrbx8sdg40000gn/T/ipykernel_85733/3870981645.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['completion_date'] = pd.to_datetime(df['protocolSection.statusModule.completionDateStruct.date'], errors='coerce', infer_datetime_format=True)
/var/folders/s8/5rrw77y553vc4scyrbx8sdg40000gn/T/ipykernel_85733/3870981645.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['start_date'] = pd.to_datetime(df['protocolSection.statusModule.startDateStruct.date'], errors='coerce', infer_datetime_format=True)
/var/folders/s8/5rrw77y5

The number of data from January 2008 to December 31th 2021: 3236


In [57]:
filtered_df.to_csv('PaperFiltered_v1.csv', index=False)
print("Data saved to 'PaperFiltered_v1.csv'.")

Data saved to 'PaperFiltered_v1.csv'.


In [55]:
import ast

def extract_intervention_types(interventions):
    # interventions가 문자열이면 리스트로 변환 시도
    if isinstance(interventions, str):
        try:
            interventions = ast.literal_eval(interventions)  # 문자열을 리스트로 변환
        except (ValueError, SyntaxError):
            return []  # 변환할 수 없는 경우 빈 리스트 반환
    
    # interventions가 리스트이면 'type' 값 추출
    if isinstance(interventions, list):
        return [item['type'] for item in interventions if isinstance(item, dict) and 'type' in item]
    
    return []  # 리스트가 아닌 경우 빈 리스트 반환
df['InterventionalType'] = df['protocolSection.armsInterventionsModule.interventions'].apply(extract_intervention_types)

# 필터링된 데이터가 리스트에 'BIOLOGICAL' 또는 'DRUG'가 포함된 경우를 찾음
filtered_df = df[df['InterventionalType'].apply(lambda x: 'BIOLOGICAL' in x or 'DRUG' in x)]

# 필터링된 데이터의 개수 출력
print(filtered_df.shape[0])


26236
